In [2]:
import tweepy
import sys
import jsonpickle
import time
from datetime import datetime
from datetime import timedelta

In [ ]:
consumer_key = '....'
consumer_secret = '...'
access_token = '...'
access_token_secret = '...'

since =(datetime.now() + timedelta(days=-1)).strftime("%Y-%m-%d")
#qry='pindah ibu kota -filter:retweets -filter:replies -filter:links -filter:images -filter:videos'
qry='kebakaran hutan -filter:links -filter:images -filter:videos '+'since:'+since
maxTweets = 600 # Isi sembarang nilai sesuai kebutuhan anda
tweetsPerQry = 100  # Jangan isi lebih dari 100, ndak boleh oleh Twitter
fName=since+'_Tweets.json' # Nama File hasil Crawling

auth = tweepy.AppAuthHandler(consumer_key,consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

if (not api):
    sys.exit('Autentikasi gagal, mohon cek "Consumer Key" & "Consumer Secret" Twitter anda')

sinceId, max_id, tweetCount = None, -1, 0 

print("Mulai mengunduh maksimum {0} tweets".format(maxTweets))
with open(fName,'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry, tweet_mode='extended')
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,since_id=sinceId, tweet_mode='extended')
            else:
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1), tweet_mode='extended')
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1),since_id=sinceId, tweet_mode='extended')
            if not new_tweets:
                print('Tidak ada lagi Tweet ditemukan dengan Query="{0}"'.format(qry));break
            for tweet in new_tweets:
                if(tweet.created_at.date() < datetime.now().date()) :
                    f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
            tweetCount+=len(new_tweets)
            sys.stdout.write("\r");sys.stdout.write("Jumlah Tweets telah tersimpan: %.0f" %tweetCount);sys.stdout.flush()
            max_id=new_tweets[-1].id
        except tweepy.TweepError as e:
            print("some error : " + str(e));break # Aya error, keluar
print ('\nSelesai! {0} tweets tersimpan di "{1}"'.format(tweetCount,fName))